In [1]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# ============================================================================
# CONFIGURATION
# ============================================================================
DATA_FILE = '../data/token_transfers.csv'  # <-- PUT YOUR USDT TRANSFER FILE HERE

# Define investor categories based on transaction value
WHALE_THRESHOLD = 100000      # > $100k
MEDIUM_THRESHOLD = 10000      # $10k - $100k
SMALL_THRESHOLD = 1000        # $1k - $10k
# Retail is < $1k

CRISIS_DATE = '2022-05-10'
# ============================================================================

df = pd.read_csv(DATA_FILE)

# Prepare data
df['datetime'] = pd.to_datetime(df['time_stamp'], unit='s')
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df = df[df['value'] > 0].copy()

print(f"✓ Loaded {len(df):,} transactions")
print(f"✓ Date range: {df['datetime'].min()} to {df['datetime'].max()}")

# Categorize investors by transaction size
def categorize_investor(value):
    if value >= WHALE_THRESHOLD:
        return 'Whale (>$100k)'
    elif value >= MEDIUM_THRESHOLD:
        return 'Medium ($10k-$100k)'
    elif value >= SMALL_THRESHOLD:
        return 'Small ($1k-$10k)'
    else:
        return 'Retail (<$1k)'

print("\nCategorizing investors by transaction size...")
df['investor_type'] = df['value'].apply(categorize_investor)

# Print distribution
print("\nInvestor Distribution:")
print(df['investor_type'].value_counts().sort_index())
print("\nValue Distribution:")
for inv_type in ['Whale (>$100k)', 'Medium ($10k-$100k)', 'Small ($1k-$10k)', 'Retail (<$1k)']:
    total = df[df['investor_type'] == inv_type]['value'].sum()
    print(f"  {inv_type}: ${total:,.2f}")

# ============================================================================
# CHART 1: Transaction VALUE Timeline (Hourly)
# ============================================================================
print("\n📈 Creating Chart 1: Transaction Value Timeline (Hourly)...")

# Group by hour and investor type - SUM of values
hourly_value_by_type = df.groupby([df['datetime'].dt.floor('h'), 'investor_type'])['value'].sum().reset_index()
hourly_value_by_type.columns = ['datetime', 'investor_type', 'total_value']

fig, ax = plt.subplots(figsize=(18, 9))

# Define colors for each investor type
colors = {
    'Whale (>$100k)': '#e74c3c',      # Red
    'Medium ($10k-$100k)': '#f39c12',  # Orange
    'Small ($1k-$10k)': '#3498db',     # Blue
    'Retail (<$1k)': '#2ecc71'         # Green
}

# Plot each investor type
for inv_type in ['Whale (>$100k)', 'Medium ($10k-$100k)', 'Small ($1k-$10k)', 'Retail (<$1k)']:
    data = hourly_value_by_type[hourly_value_by_type['investor_type'] == inv_type]
    if len(data) > 0:
        ax.plot(data['datetime'], data['total_value'], 
                linewidth=3, label=inv_type, color=colors[inv_type], alpha=0.8)

# Mark crisis date
crisis_dt = pd.to_datetime(CRISIS_DATE)
ax.axvline(crisis_dt, color='red', linestyle='--', linewidth=3, alpha=0.7, 
           label='May 10, 2022 (UST Collapse)')

ax.set_title('USDT: Who Moved The Most Money? Transaction Value by Investor Type', 
             fontsize=20, fontweight='bold', pad=20)
ax.set_xlabel('Date/Time', fontsize=14, fontweight='bold')
ax.set_ylabel('Total Transaction Value ($)', fontsize=14, fontweight='bold')
ax.legend(fontsize=12, loc='upper left', framealpha=0.9)
ax.grid(True, alpha=0.3)
ax.set_yscale('log')  # Log scale to see all categories
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('usdt_whale_vs_retail_value_timeline.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: usdt_whale_vs_retail_value_timeline.png")
plt.close()

# ============================================================================
# CHART 2: Transaction COUNT Timeline (Hourly)
# ============================================================================
print("\n📊 Creating Chart 2: Transaction Count Timeline (Hourly)...")

# Group by hour and investor type - COUNT
hourly_count_by_type = df.groupby([df['datetime'].dt.floor('h'), 'investor_type']).size().reset_index(name='count')
hourly_count_by_type.columns = ['datetime', 'investor_type', 'count']

fig, ax = plt.subplots(figsize=(18, 9))

# Plot each investor type
for inv_type in ['Whale (>$100k)', 'Medium ($10k-$100k)', 'Small ($1k-$10k)', 'Retail (<$1k)']:
    data = hourly_count_by_type[hourly_count_by_type['investor_type'] == inv_type]
    if len(data) > 0:
        ax.plot(data['datetime'], data['count'], 
                linewidth=3, label=inv_type, color=colors[inv_type], alpha=0.8)

ax.axvline(crisis_dt, color='red', linestyle='--', linewidth=3, alpha=0.7, 
           label='May 10, 2022 (UST Collapse)')

ax.set_title('USDT: Who Panicked First? Transaction Count by Investor Type', 
             fontsize=20, fontweight='bold', pad=20)
ax.set_xlabel('Date/Time', fontsize=14, fontweight='bold')
ax.set_ylabel('Number of Transactions', fontsize=14, fontweight='bold')
ax.legend(fontsize=12, loc='upper left', framealpha=0.9)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('usdt_whale_vs_retail_count_timeline.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: usdt_whale_vs_retail_count_timeline.png")
plt.close()

# ============================================================================
# CHART 3: Combined View (2 panels)
# ============================================================================
print("\n📈 Creating Chart 3: Combined Timeline (Value + Count)...")

fig, axes = plt.subplots(2, 1, figsize=(20, 12))

# Panel 1: Transaction VALUE
ax1 = axes[0]
for inv_type in ['Whale (>$100k)', 'Medium ($10k-$100k)', 'Small ($1k-$10k)', 'Retail (<$1k)']:
    data = hourly_value_by_type[hourly_value_by_type['investor_type'] == inv_type]
    if len(data) > 0:
        ax1.plot(data['datetime'], data['total_value'], 
                linewidth=3, label=inv_type, color=colors[inv_type], alpha=0.8)

ax1.axvline(crisis_dt, color='red', linestyle='--', linewidth=3, alpha=0.7, 
           label='May 10, 2022')
ax1.set_title('Transaction VALUE by Investor Type', fontsize=16, fontweight='bold')
ax1.set_ylabel('Total Transaction Value ($)', fontsize=12, fontweight='bold')
ax1.legend(fontsize=11, loc='upper left', framealpha=0.9)
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')
ax1.tick_params(axis='x', labelbottom=False)

# Panel 2: Transaction COUNT
ax2 = axes[1]
for inv_type in ['Whale (>$100k)', 'Medium ($10k-$100k)', 'Small ($1k-$10k)', 'Retail (<$1k)']:
    data = hourly_count_by_type[hourly_count_by_type['investor_type'] == inv_type]
    if len(data) > 0:
        ax2.plot(data['datetime'], data['count'], 
                linewidth=3, label=inv_type, color=colors[inv_type], alpha=0.8)

ax2.axvline(crisis_dt, color='red', linestyle='--', linewidth=3, alpha=0.7)
ax2.set_title('Transaction COUNT by Investor Type', fontsize=16, fontweight='bold')
ax2.set_xlabel('Date/Time', fontsize=12, fontweight='bold')
ax2.set_ylabel('Number of Transactions', fontsize=12, fontweight='bold')
ax2.legend(fontsize=11, loc='upper left', framealpha=0.9)
ax2.grid(True, alpha=0.3)
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)

plt.suptitle('USDT Whale vs Retail Analysis: Value & Count Timeline', 
             fontsize=22, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('usdt_whale_vs_retail_combined.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: usdt_whale_vs_retail_combined.png")
plt.close()

# ============================================================================
# SUMMARY STATISTICS
# ============================================================================
print("\n" + "="*70)
print("WHALE VS RETAIL ANALYSIS SUMMARY")
print("="*70)

# Split before and after crisis
before_crisis = df[df['datetime'] < crisis_dt]
after_crisis = df[df['datetime'] >= crisis_dt]

print("\nBEFORE Crisis (Before May 10, 2022):")
for inv_type in ['Whale (>$100k)', 'Medium ($10k-$100k)', 'Small ($1k-$10k)', 'Retail (<$1k)']:
    count = len(before_crisis[before_crisis['investor_type'] == inv_type])
    total_value = before_crisis[before_crisis['investor_type'] == inv_type]['value'].sum()
    avg_value = before_crisis[before_crisis['investor_type'] == inv_type]['value'].mean()
    print(f"  {inv_type:25} | Count: {count:6,} | Total: ${total_value:15,.2f} | Avg: ${avg_value:10,.2f}")

print("\nAFTER Crisis (After May 10, 2022):")
for inv_type in ['Whale (>$100k)', 'Medium ($10k-$100k)', 'Small ($1k-$10k)', 'Retail (<$1k)']:
    count = len(after_crisis[after_crisis['investor_type'] == inv_type])
    total_value = after_crisis[after_crisis['investor_type'] == inv_type]['value'].sum()
    avg_value = after_crisis[after_crisis['investor_type'] == inv_type]['value'].mean()
    print(f"  {inv_type:25} | Count: {count:6,} | Total: ${total_value:15,.2f} | Avg: ${avg_value:10,.2f}")

print("\nCHANGE (Before → After):")
for inv_type in ['Whale (>$100k)', 'Medium ($10k-$100k)', 'Small ($1k-$10k)', 'Retail (<$1k)']:
    count_before = len(before_crisis[before_crisis['investor_type'] == inv_type])
    count_after = len(after_crisis[after_crisis['investor_type'] == inv_type])
    count_change = ((count_after - count_before) / count_before * 100) if count_before > 0 else 0
    
    value_before = before_crisis[before_crisis['investor_type'] == inv_type]['value'].sum()
    value_after = after_crisis[after_crisis['investor_type'] == inv_type]['value'].sum()
    value_change = ((value_after - value_before) / value_before * 100) if value_before > 0 else 0
    
    print(f"  {inv_type:25} | Count: {count_change:+7.1f}% | Value: {value_change:+7.1f}%")

print("\n" + "="*70)
print("KEY INSIGHTS:")
print("="*70)

# Who moved first?
whale_before = before_crisis[before_crisis['investor_type'] == 'Whale (>$100k)']['value'].sum()
whale_after = after_crisis[after_crisis['investor_type'] == 'Whale (>$100k)']['value'].sum()
whale_change = ((whale_after - whale_before) / whale_before * 100) if whale_before > 0 else 0

retail_before = before_crisis[before_crisis['investor_type'] == 'Retail (<$1k)']['value'].sum()
retail_after = after_crisis[after_crisis['investor_type'] == 'Retail (<$1k)']['value'].sum()
retail_change = ((retail_after - retail_before) / retail_before * 100) if retail_before > 0 else 0

print(f"\n1. Whale activity changed by {whale_change:+.1f}%")
print(f"2. Retail activity changed by {retail_change:+.1f}%")

if whale_change > retail_change:
    print(f"\n→ Whales increased activity MORE than retail (+{whale_change - retail_change:.1f}% difference)")
    print("   This suggests whales moved TO USDT faster (flight to safety)")
else:
    print(f"\n→ Retail increased activity MORE than whales (+{retail_change - whale_change:.1f}% difference)")
    print("   This suggests retail panic drove the volume spike")

print("\n" + "="*70)
print("\n✅ All charts created successfully!")
print("\nFiles generated:")
print("  1. usdt_whale_vs_retail_value_timeline.png  (Transaction VALUE)")
print("  2. usdt_whale_vs_retail_count_timeline.png  (Transaction COUNT)")
print("  3. usdt_whale_vs_retail_combined.png        (Both in one)")
print("="*70)

✓ Loaded 5,277,372 transactions
✓ Date range: 2022-04-28 00:30:15 to 2022-05-25 18:47:44

Categorizing investors by transaction size...

Investor Distribution:
investor_type
Medium ($10k-$100k)    1031785
Retail (<$1k)          1998527
Small ($1k-$10k)       1649351
Whale (>$100k)          597709
Name: count, dtype: int64

Value Distribution:
  Whale (>$100k): $15,142,048,703,255.76
  Medium ($10k-$100k): $35,047,581,105.25
  Small ($1k-$10k): $6,002,951,561.14
  Retail (<$1k): $616,424,009.34

📈 Creating Chart 1: Transaction Value Timeline (Hourly)...
  ✓ Saved: usdt_whale_vs_retail_value_timeline.png

📊 Creating Chart 2: Transaction Count Timeline (Hourly)...
  ✓ Saved: usdt_whale_vs_retail_count_timeline.png

📈 Creating Chart 3: Combined Timeline (Value + Count)...
  ✓ Saved: usdt_whale_vs_retail_combined.png

WHALE VS RETAIL ANALYSIS SUMMARY

BEFORE Crisis (Before May 10, 2022):
  Whale (>$100k)            | Count: 164,857 | Total: $208,761,086,892.11 | Avg: $1,266,316.18
  Medium 